* 참고자료 1 : https://webnautes.tistory.com/779
* 참고자료 2 : https://webnautes.tistory.com/1239
* 추가 공부 자료 1 (동적 페이지 다루기) : https://luminitworld.tistory.com/88 
* 추가 공부 자료 2 (동적 페이지 다루기) : https://velog.io/@lybin10/%EB%8F%99%EC%A0%81-%EC%9B%B9-%ED%8E%98%EC%9D%B4%EC%A7%80-%ED%81%AC%EB%A1%A4%EB%A7%81

In [1]:
import os
import pandas as pd
import urllib.request
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

## 🔹 코드 ver.1

* 정적 페이지 : 크롤링+스크레이핑 적용
* 1페이지만 동작

In [2]:
# 이미지 저장할 폴더 만들기
if not os.path.isdir('Book_img'):
    os.mkdir('Book_img')

url = "https://book.naver.com/bestsell/bestseller_list.nhn"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(30)


# 네이버의 베스트셀러 웹페이지를 가져옵니다.
driver.get(url)
bsObject = BeautifulSoup(driver.page_source, 'html.parser')

book_info_dict = {'title':[], 'author':[], 'url':[], 'price':[]}
    
    
# 책의 상세 웹페이지 주소를 추출하여 리스트에 저장합니다.
book_page_urls = []
for index in range(0, 25):
    dl_data = bsObject.find('dt', {'id':"book_title_"+str(index)})
    link = dl_data.select('a')[0].get('href')
    book_page_urls.append(link)


# 메타 정보와 본문에서 필요한 정보를 추출합니다.
for book_page_url in book_page_urls:

    driver.get(book_page_url)
    bsObject = BeautifulSoup(driver.page_source, 'html.parser')

    title = bsObject.find('div', 'bookTitle_title_area__fspvB').h2.text
    book_info_dict['title'].append(title)

    author = bsObject.find('div', 'bookTitle_info_content__iHfCC').text
    book_info_dict['author'].append(author)

    image = bsObject.find('meta', {'property':'og:image'}).get('content')

    if ':' in title:
        img_title = title.replace(' : ', '_')
        save_img_name = f'Book_img/{img_title}.jpg' 
    else : 
        save_img_name = f'Book_img/{title}.jpg' 
    urllib.request.urlretrieve(image, save_img_name)

    url = bsObject.find('meta', {'property':'og:url'}).get('content')
    book_info_dict['url'].append(url)

    price = bsObject.find('div', 'bookSeller_price__opK_s').text
    book_info_dict['price'].append(price)

    print(title, author, image, url, price)
    print('\n')


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 12.0MB/s]
C:\Users\Family\AppData\Local\Temp\ipykernel_25696\225301823.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


헤어질 결심 각본 정서경,박찬욱 https://shopping-phinf.pstatic.net/main_3355622/33556223644.20230214162653.jpg https://search.shopping.naver.com/book/catalog/33556223644 13,500원


흔한남매 11 흔한남매 (원작),백난도 https://shopping-phinf.pstatic.net/main_3361369/33613699623.20230218110217.jpg https://search.shopping.naver.com/book/catalog/33613699623 13,050원


불편한 편의점 김호연 https://shopping-phinf.pstatic.net/main_3244499/32444990070.20230218110501.jpg https://search.shopping.naver.com/book/catalog/32444990070 12,600원


역행자 자청 https://shopping-phinf.pstatic.net/main_3255028/32550285396.20221004114149.jpg https://search.shopping.naver.com/book/catalog/32550285396 15,750원


계속 가보겠습니다 임은정 https://shopping-phinf.pstatic.net/main_3347482/33474829621.20221019124110.jpg https://search.shopping.naver.com/book/catalog/33474829621 16,200원


마법천자문 53 유대영 https://shopping-phinf.pstatic.net/main_3343529/33435295780.20221019135216.jpg https://search.shopping.naver.com/book/catalog/33435295780 12,600원


어서 오세요, 휴남동 서점입니다 황보름 http

In [3]:
book_info_df = pd.DataFrame(book_info_dict)
book_info_df

,title,author,url,price
0,헤어질 결심 각본,"정서경,박찬욱",https://search.shopping.naver.com/book/catalog...,"13,500원"
1,흔한남매 11,"흔한남매 (원작),백난도",https://search.shopping.naver.com/book/catalog...,"13,050원"
2,불편한 편의점,김호연,https://search.shopping.naver.com/book/catalog...,"12,600원"
3,역행자,자청,https://search.shopping.naver.com/book/catalog...,"15,750원"
4,계속 가보겠습니다,임은정,https://search.shopping.naver.com/book/catalog...,"16,200원"
5,마법천자문 53,유대영,https://search.shopping.naver.com/book/catalog...,"12,600원"
6,"어서 오세요, 휴남동 서점입니다",황보름,https://search.shopping.naver.com/book/catalog...,"13,500원"
7,이상한 과자 가게 전천당 15,히로시마 레이코,https://search.shopping.naver.com/book/catalog...,"11,700원"
8,작별인사,김영하,https://search.shopping.naver.com/book/catalog...,"12,600원"
9,밥 프록터 부의 확신,밥 프록터,https://search.shopping.naver.com/book/catalog...,"14,400원"


In [4]:
book_info_df.to_excel('book_info.xlsx', encoding='utf-8', index=False)

## 🔸코드 ver.2

* 동적 페이지 : 크롤링+스크레이핑 적용
* 전체 페이지에서 동작

In [9]:
# 이미지 저장할 폴더 만들기
if not os.path.isdir('Book_img'):
    os.mkdir('Book_img')

    
url = "https://book.naver.com/bestsell/bestseller_list.nhn"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(40)


book_info_dict = {'title':[], 'author':[], 'url':[], 'price':[]}


# '네이버 : 베스트셀러 웹페이지' 가져오기
driver.get(url)
bsObject = BeautifulSoup(driver.page_source, 'html.parser')


for index_1 in range(0, 6):
    if index_1 != 0:
        driver.find_element(By.XPATH, f'//*[@id="section_bestseller"]/div[4]/a[{index_1}]').click()
        print('클릭 완료!')

    # 책의 상세 웹페이지 주소를 추출하여 리스트에 저장합니다.
    for index_2 in range(0, 25):
        driver.find_element(By.XPATH, f'//*[@id="book_title_{index_2}"]/a').click()
        driver.switch_to.window(driver.window_handles[1])
        
        bsObject = BeautifulSoup(driver.page_source, 'html.parser')
        
        try:
            title = bsObject.find('div', 'bookTitle_title_area__fspvB').h2.text
        except AttributeError as e:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            driver.get('https://book.naver.com/bestsell/bestseller_list.nhn')
            driver.find_element(By.XPATH, f'//*[@id="section_bestseller"]/div[4]/a[{index_1}]').click()
            
            driver.find_element(By.XPATH, f'//*[@id="book_title_{index_2}"]/a').click()
            driver.switch_to.window(driver.window_handles[1])
            
            bsObject = BeautifulSoup(driver.page_source, 'html.parser')
            title = bsObject.find('div', 'bookTitle_title_area__fspvB').h2.text
        book_info_dict['title'].append(title)
            

        author = bsObject.find('div', 'bookTitle_info_content__iHfCC').text
        book_info_dict['author'].append(author)

        image = bsObject.find('meta', {'property':'og:image'}).get('content')

        if ':' in title:
            img_title = title.replace(': ', '_')
            save_img_name = f'Book_img/{img_title}.jpg' 
        else : 
            save_img_name = f'Book_img/{title}.jpg' 
        urllib.request.urlretrieve(image, save_img_name)

        url = bsObject.find('meta', {'property':'og:url'}).get('content')
        book_info_dict['url'].append(url)

        try:
            price = bsObject.find('div', 'bookSeller_price__opK_s').text
        except AttributeError as e:
            driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/div[1]/div[3]/div/a').click()
            price = driver.find_element(By.XPATH, '//*[@id="book_section-seller"]/div[2]/ul/li[1]/div/a/span[3]/strong').text
        book_info_dict['price'].append(price)

        print(title, author, image, url, price)
        print('\n')
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    

C:\Users\Family\AppData\Local\Temp\ipykernel_25696\2518494460.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


헤어질 결심 각본 정서경,박찬욱 https://shopping-phinf.pstatic.net/main_3355622/33556223644.20230214162653.jpg https://search.shopping.naver.com/book/catalog/33556223644 13,500원


흔한남매 11 흔한남매 (원작),백난도 https://shopping-phinf.pstatic.net/main_3361369/33613699623.20230218110217.jpg https://search.shopping.naver.com/book/catalog/33613699623 13,050원


불편한 편의점 김호연 https://shopping-phinf.pstatic.net/main_3244499/32444990070.20230218110501.jpg https://search.shopping.naver.com/book/catalog/32444990070 12,600원


역행자 자청 https://shopping-phinf.pstatic.net/main_3255028/32550285396.20221004114149.jpg https://search.shopping.naver.com/book/catalog/32550285396 15,750원


계속 가보겠습니다 임은정 https://shopping-phinf.pstatic.net/main_3347482/33474829621.20221019124110.jpg https://search.shopping.naver.com/book/catalog/33474829621 16,200원


마법천자문 53 유대영 https://shopping-phinf.pstatic.net/main_3343529/33435295780.20221019135216.jpg https://search.shopping.naver.com/book/catalog/33435295780 12,600원


어서 오세요, 휴남동 서점입니다 황보름 http

어느 날, 내 죽음에 네가 들어왔다  세이카 료겐 https://shopping-phinf.pstatic.net/main_3247335/32473357669.20230221082038.jpg https://search.shopping.naver.com/book/catalog/32473357669 13,500원


오늘 밤, 세계에서 이 사랑이 사라진다 해도 이치조 미사키 https://shopping-phinf.pstatic.net/main_3247607/32476078068.20230207165225.jpg https://search.shopping.naver.com/book/catalog/32476078068 12,600원


클릭 완료!
2022 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1, 2, 3급)(상) 최태성 https://shopping-phinf.pstatic.net/main_3249237/32492375912.20230103163058.jpg https://search.shopping.naver.com/book/catalog/32492375912 13,500원


명탐정 코난 : 경찰학교 편 - 하 아오야마 고쇼 https://shopping-phinf.pstatic.net/main_3244497/32444971893.20221228074011.jpg https://search.shopping.naver.com/book/catalog/32444971893 4,950원


한 개의 기쁨이 천 개의 슬픔을 이긴다 조우성 https://shopping-phinf.pstatic.net/main_3317994/33179949641.20230117163510.jpg https://search.shopping.naver.com/book/catalog/33179949641 13,320원


금쪽이들의 진짜 마음속 오은영 https://shopping-phinf.pstatic.net/main_3248500/32485005828.202210191236

눈물 한 방울  이어령 https://shopping-phinf.pstatic.net/main_3309109/33091097628.20230217105249.jpg https://search.shopping.naver.com/book/catalog/33091097628 14,220원


예의 없는 친구들을 대하는 슬기로운 말하기 사전 김원아 https://shopping-phinf.pstatic.net/main_3246657/32466572094.20230216071059.jpg https://search.shopping.naver.com/book/catalog/32466572094 11,700원


만화 악역의 엔딩은 죽음뿐 4 한정판 수월 https://shopping-phinf.pstatic.net/main_3364247/33642474644.20221019152753.jpg https://search.shopping.naver.com/book/catalog/33642474644 33,000원


클릭 완료!
돈의 심리학 모건 하우절 https://shopping-phinf.pstatic.net/main_3246701/32467012122.20230131164743.jpg https://search.shopping.naver.com/book/catalog/32467012122 17,820원


흔한남매의 흔한 호기심 6 안치현 https://shopping-phinf.pstatic.net/main_3260892/32608920625.20230117165614.jpg https://search.shopping.naver.com/book/catalog/32608920625 11,700원


미드나잇 라이브러리 매트 헤이그 https://shopping-phinf.pstatic.net/main_3243818/32438188779.20230214162555.jpg https://search.shopping.naver.com/book/catalog/32438188

만복이네 떡집 김리리 https://shopping-phinf.pstatic.net/main_3244047/32440478504.20230216071019.jpg https://search.shopping.naver.com/book/catalog/32440478504 9,900원


초등 사자소학 송재환 https://shopping-phinf.pstatic.net/main_3244404/32444048627.20230214162907.jpg https://search.shopping.naver.com/book/catalog/32444048627 12,420원


고양이 해결사 깜냥 1 홍민정 https://shopping-phinf.pstatic.net/main_3245816/32458162983.20230219103439.jpg https://search.shopping.naver.com/book/catalog/32458162983 9,900원


총 균 쇠 재레드 다이아몬드 https://shopping-phinf.pstatic.net/main_3245316/32453160133.20230207165745.jpg https://search.shopping.naver.com/book/catalog/32453160133 25,200원


저만치 혼자서 김훈 https://shopping-phinf.pstatic.net/main_3268297/32682971623.20230223070954.jpg https://search.shopping.naver.com/book/catalog/32682971623 13,500원




In [10]:
book_info_df = pd.DataFrame(book_info_dict)
book_info_df

,title,author,url,price
0,헤어질 결심 각본,"정서경,박찬욱",https://search.shopping.naver.com/book/catalog...,"13,500원"
1,흔한남매 11,"흔한남매 (원작),백난도",https://search.shopping.naver.com/book/catalog...,"13,050원"
2,불편한 편의점,김호연,https://search.shopping.naver.com/book/catalog...,"12,600원"
3,역행자,자청,https://search.shopping.naver.com/book/catalog...,"15,750원"
4,계속 가보겠습니다,임은정,https://search.shopping.naver.com/book/catalog...,"16,200원"
...,...,...,...,...
145,만복이네 떡집,김리리,https://search.shopping.naver.com/book/catalog...,"9,900원"
146,초등 사자소학,송재환,https://search.shopping.naver.com/book/catalog...,"12,420원"
147,고양이 해결사 깜냥 1,홍민정,https://search.shopping.naver.com/book/catalog...,"9,900원"
148,총 균 쇠,재레드 다이아몬드,https://search.shopping.naver.com/book/catalog...,"25,200원"


In [11]:
book_info_df.to_excel('book_info_2.xlsx', encoding='utf-8', index=False)

In [12]:
pd.read_excel('book_info_2.xlsx')

,title,author,url,price
0,헤어질 결심 각본,"정서경,박찬욱",https://search.shopping.naver.com/book/catalog...,"13,500원"
1,흔한남매 11,"흔한남매 (원작),백난도",https://search.shopping.naver.com/book/catalog...,"13,050원"
2,불편한 편의점,김호연,https://search.shopping.naver.com/book/catalog...,"12,600원"
3,역행자,자청,https://search.shopping.naver.com/book/catalog...,"15,750원"
4,계속 가보겠습니다,임은정,https://search.shopping.naver.com/book/catalog...,"16,200원"
...,...,...,...,...
145,만복이네 떡집,김리리,https://search.shopping.naver.com/book/catalog...,"9,900원"
146,초등 사자소학,송재환,https://search.shopping.naver.com/book/catalog...,"12,420원"
147,고양이 해결사 깜냥 1,홍민정,https://search.shopping.naver.com/book/catalog...,"9,900원"
148,총 균 쇠,재레드 다이아몬드,https://search.shopping.naver.com/book/catalog...,"25,200원"


## 👉중복 title 가진 데이터 확인하기

In [13]:
book_info_df[book_info_df['title'].duplicated()]

,title,author,url,price
25,헤어질 결심 각본,"정서경,박찬욱",https://search.shopping.naver.com/book/catalog...,"13,500원"
60,원펀맨(One Punch Man) 25,ONE,https://search.shopping.naver.com/book/catalog...,"4,950원"
118,한 개의 기쁨이 천 개의 슬픔을 이긴다,조우성,https://search.shopping.naver.com/book/catalog...,"13,320원"
